In [1]:
#Import dependencies.
import pandas as pd
import gmaps
import gmaps.datasets
import requests

#Import the API key.
from config import g_key

In [2]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
city_data_df.head()

,City_ID,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain inches last 3 hours,Snow inches last 3 hours
0,0,0,Ushuaia,-54.80,-68.30,46.40,87,90,20.80,AR,2020-02-23 21:46:25,overcast clouds,0.0,0.0
1,1,1,Atuona,-9.80,-139.03,81.61,76,8,13.73,PF,2020-02-23 21:43:44,clear sky,0.0,0.0
2,2,2,Lichinga,-13.31,35.24,72.52,92,66,2.80,MZ,2020-02-23 21:46:25,broken clouds,0.0,0.0
3,3,3,Yenagoa,4.92,6.26,82.53,77,100,5.57,NG,2020-02-23 21:46:26,overcast clouds,0.0,0.0
4,4,4,Punta Arenas,-53.15,-70.92,55.40,62,75,31.09,CL,2020-02-23 21:46:26,broken clouds,0.0,0.0


In [4]:
Weymouth = city_data_df[city_data_df['City'].str.contains("Weymouth")]
Weymouth

,City_ID,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain inches last 3 hours,Snow inches last 3 hours
29,29,29,Weymouth,42.22,-70.94,54.0,27,1,10.29,US,2020-02-23 21:43:55,clear sky,0.0,0.0


In [5]:
Albany = city_data_df[city_data_df['City'].str.contains("Albany")]
Albany

,City_ID,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain inches last 3 hours,Snow inches last 3 hours
62,62,62,Albany,42.6,-73.97,52.0,29,75,5.82,US,2020-02-23 21:46:39,broken clouds,0.0,0.0


In [6]:
Jamestown = city_data_df[city_data_df['City'].str.contains("Jamestown")]
Jamestown

,City_ID,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain inches last 3 hours,Snow inches last 3 hours
53,53,53,Jamestown,42.1,-79.24,48.99,26,1,16.11,US,2020-02-23 21:46:37,clear sky,0.0,0.0


In [7]:
Bethel = city_data_df[city_data_df['City'].str.contains("Bethel")]
Bethel

,City_ID,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain inches last 3 hours,Snow inches last 3 hours
66,66,66,Bethel,41.37,-73.41,54.0,20,5,9.17,US,2020-02-23 21:46:40,clear sky,0.0,0.0


In [8]:
cities=pd.concat([Bethel, Jamestown, Albany, Weymouth])
selected_cities=cities.loc[:,['City','Lat','Lng']]
selected_cities

,City,Lat,Lng
66,Bethel,41.37,-73.41
53,Jamestown,42.10,-79.24
62,Albany,42.60,-73.97
29,Weymouth,42.22,-70.94


In [9]:
# Latitude-longitude pairs
Bethel = (41.37, -73.41)
Jamestown=(42.10, -79.24)
Albany=(42.60, -73.97)
Weymouth = (42.22, -70.94)

fig = gmaps.figure()
Bethel2Jamestown = gmaps.directions_layer(
    Bethel, Jamestown, waypoints=[Albany, Weymouth],
    travel_mode='BICYCLING')
fig.add_layer(Bethel2Jamestown)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
hotel_df = cities[["City", "Country", "Max Temp","Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
66,Bethel,US,54.00,clear sky,41.37,-73.41,
53,Jamestown,US,48.99,clear sky,42.10,-79.24,
62,Albany,US,52.00,broken clouds,42.60,-73.97,
29,Weymouth,US,54.00,clear sky,42.22,-70.94,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
     # Grab the first hotel from the results and store the name.
  
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.


In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [13]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
            # max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))